In [29]:
# data sources from fangraphs.com via the pybaseball module
from pybaseball import statcast_single_game

data = statcast_single_game(747203)


In [92]:
# fangraphs weighs
# TODO: figure out a way to automate this
woba_24 = .310	
wobascale_24 = 1.272
wbb_24 = .695	
whbp_24 = .727	
w1b_24 = .892	
w2b_24 = 1.273	
w3b_24  = 1.617	
whr_24 = 2.091	
runsb_24 = .200	
runcs_24 = -.400	
rpa_24 = .115	
rw = 9.584	
cfip = 3.150
pf_dbacks = 1.0

In [ ]:
# To calculate Batting Runs Above Average you only need to know three things about a player and several things about the league in general. You need the player’s wOBA, PA, and home park factor and you need League Average wOBA (lgwOBA), the wOBA Scale, MLB R/PA (lgR/PA), and the specific league (AL or NL) wRC and PA for non-pitchers. League wOBA, wOBA Scale, and R/PA can be found here. Park Factors can be found here and wRC and PA for the leagues can be found using the leaderboards.

# The first step is to find Weighted Runs Above Average (wRAA) from the player’s wOBA, or you may simply find their wRAA on FanGraphs. To calculate their wRAA, do the following:


# wRAA = ((wOBA – lgwOBA)/wOBA Scale) * PA
wRAA = 0
wOBA = 0
lgwOBA = .310	
wOBA_scale = 1.278	
PA = 183

data 

# wRAA is simply a non-adjusted Batting Runs. To adjust wRAA for park and league, you do the following with the park factor expressed as a decimal (i.e. 0.95 for 95):

Batting Runs = wRAA + (lgR/PA – (PF*lgR/PA))*PA + (lgR/PA – (AL or NL non-pitcher wRC/PA))*PA

Essentially, you are talking wRAA and adding or subtracting in runs based on their home park and their league.

In [31]:
import pandas as pd

In [30]:
games_data = [statcast_single_game(game) for game in games]
    

NameError: name 'pd' is not defined

In [33]:
data = pd.concat(games_data)




In [18]:
from pybaseball import cache 
cache.enable()

In [28]:
# group by game_id
games = list(set(data['game_pk']))


In [75]:
from pybaseball import statcast, statcast_batter, playerid_lookup

# data_latest = statcast(start_dt='2024-03-28', end_dt='2024-05-21', team="AZ")
# playerid_lookup('carroll', 'corbin') # 682998
cncl = statcast_batter('2024-03-28', '2024-05-21',682998)

Gathering Player Data


In [70]:
# data = pd.concat([data, data_latest])
data = data_latest.copy()

In [46]:
data.to_csv('./dbacks_0521.csv')

In [79]:
# data['events'].value_counts()
cncl['events'].value_counts()
# TODO: check these for IBBs

field_out                    95
strikeout                    38
single                       26
walk                         20
force_out                     5
double                        4
triple                        2
hit_by_pitch                  2
fielders_choice               2
field_error                   2
home_run                      2
grounded_into_double_play     1
sac_fly                       1
double_play                   1
Name: events, dtype: int64

In [87]:
# cncl = data[data['player_name'] == 'Carroll, Corbin']  
cncl_events = cncl['events']
# plate appearences
pa = cncl['events'].count()
bb = cncl_events[cncl_events == 'walk'].count()
hbp = cncl_events[cncl_events == 'hit_by_pitch'].count()
b1 = cncl_events[cncl_events == 'single'].count()
b2 = cncl_events[cncl_events == 'double'].count()
b3 = cncl_events[cncl_events == 'triple'].count()
hr  = cncl_events[cncl_events == 'home_run'].count()
# TODO implement IBB
# ibb  = cncl_events['walk']
ibb  = 0
sf  = cncl_events[cncl_events == 'sac_fly'].count()
# TODO implement sac
error = cncl_events[cncl_events == 'field_error'].count() 
ab  = pa - bb - sf - error

woba = (
    bb * wbb_24  
    + hbp * whbp_24 
    + b1 * w1b_24 
    + b2 * w2b_24 
    + b3 * w3b_24 
    + hr * whr_24
)/(ab + bb - ibb + sf + hbp)

print(woba)



0.254


In [116]:
def woba_top(event, pos):
    top = 0
    bottom = 0
    if event == 'walk':
        top = wbb_24
        bottom = 1
    if event == 'hit_by_pitch':
        top = whbp_24
        bottom = 1
    if event == 'single':
        top = w1b_24
        bottom = 1
    if event == 'double':
        top = w2b_24
        bottom = 1
    if event == 'triple':
        top = w3b_24
        bottom = 1
    if event == 'home_run':
        top = whr_24
        bottom = 1
    if event == 'sac_fly':
        top = 0
        bottom = 1
    
    if pos == 0:
        return(top)
    if pos == 1:
        return(bottom)
    
    


In [119]:
cncl['woba_top'] = [woba_top(e, 0) for e in cncl['events']]
cncl['woba_bottom'] = [woba_top(e, 1) for e in cncl['events']]
cncl['woba_top_diff'] = cncl['woba_top'] - cncl['woba_value']

In [86]:
print(wbb_24, whbp_24, w1b_24, w2b_24, w3b_24, whr_24)
print(bb, hbp, b1, b2, b3, hr)
print(ab, bb, ibb, hbp)
print(hbp, error, sf)

0.695 0.727 0.892 1.273 1.617 2.091
20 2 26 4 2 2
178 20 0 2
2 2 1


In [93]:
# calculating batting runs
wraa = ((woba - woba_24) / wobascale_24) * pa

wraa + (rpa_24 - (pf_dbacks*rpa_24)) * pa

-8.849056603773585

In [ ]:
# grouping 